In [11]:
import pandas as pd
import numpy as np
table_all_current = pd.read_pickle('table_all_current_7.29.19.pkl')
pd.__version__
import matplotlib.pyplot as plt

In [2]:
table_all_current.shape

(67, 1110)

In [3]:
table_all_current.tail()

,prob_default,prob_fullypaid,return_preds,loan_amnt,funded_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
id,,,,,,,,,,,,,,,,,,,,,
156077320,0.101173,0.898827,0.066045,35450,7250.0,0.1033,1.0,40000.0,26.04,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155809619,0.747000,0.253000,0.089305,40000,7925.0,0.1524,8.0,102612.0,26.85,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155879545,0.006096,0.993904,0.088831,40000,6050.0,0.1430,6.0,120000.0,9.70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156086684,0.592538,0.407462,0.178415,40000,4800.0,0.2500,1.0,135000.0,17.70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156050759,0.021257,0.978743,0.046662,40000,4350.0,0.0756,1.0,110000.0,34.72,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def expected_portfolio_return_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_expected_returns = []
    for idx in table_all_current.index:
        evenly_weighted_expected_returns.append(
            (table_all_current['return_preds'][idx]) * (avail_funds/len(table_all_current)))
    expected_portfolio_return = sum(evenly_weighted_expected_returns)/avail_funds
    return expected_portfolio_return

def rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds):
    table_all_current['shrop_ratio'] = np.nan
    for idx in table_all_current.index:
        table_all_current['shrop_ratio'][idx] = (
            (expected_portfolio_return_evenly_weighted(table_all_current, avail_funds) - 0.0188) / 
                                                        (table_all_current['prob_default'][idx]))
    table_all_current_ranked = table_all_current.sort_values(by='shrop_ratio',axis=0, ascending=False)
    return table_all_current_ranked

def recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table = table_all_current[(table_all_current['prob_default'] <= max_prob_default) & 
                             (table_all_current['return_preds'] >= min_desired_return)]
    rec_table_ranked = rank_table_by_shrop_ratio_RAR(rec_table, avail_funds)
    return rec_table_ranked

def portfolio_prob_default_evenly_weighted(table_all_current, avail_funds):
    evenly_weighted_prob_default = []
    for idx in table_all_current.index:
        evenly_weighted_prob_default.append(
            (table_all_current['prob_default'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_prob_default = sum(evenly_weighted_prob_default)/avail_funds
    return portfolio_prob_default

def portfolio_shrop_ratio_evenly_weighted(table_all_current, avail_funds):
    table_all_current_ranked = rank_table_by_shrop_ratio_RAR(table_all_current, avail_funds)
    evenly_weighted_shrop_ratio = []
    for idx in table_all_current_ranked.index:
        evenly_weighted_shrop_ratio.append(
            (table_all_current['shrop_ratio'][idx]) * (avail_funds/len(table_all_current)))
    portfolio_shrop_ratio = sum(evenly_weighted_shrop_ratio)/avail_funds
    return portfolio_shrop_ratio

def summarize_recommendation(table_all_current, max_prob_default, min_desired_return, avail_funds):
    rec_table_ranked = recommended_loans_ranked_by_shrop_RAR(table_all_current, max_prob_default, min_desired_return, avail_funds)
    port_prob_def = portfolio_prob_default_evenly_weighted(rec_table_ranked,avail_funds)
    port_exp_return = expected_portfolio_return_evenly_weighted(rec_table_ranked, avail_funds)
    port_shrop_ratio = portfolio_shrop_ratio_evenly_weighted(rec_table_ranked,avail_funds)
    max_investable = rec_table_ranked['loan_amnt'].sum() - rec_table_ranked['funded_amnt'].sum()
    return (rec_table_ranked,port_prob_def,port_exp_return,port_shrop_ratio,max_investable)

In [5]:
(rec_table_ranked,port_prob_def,
 port_exp_return,port_shrop_ratio,max_investable) = summarize_recommendation(table_all_current,
                                                                             max_prob_default=0.2,
                                                                             min_desired_return=0.07,
                                                                             avail_funds=10000)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_

In [12]:
rec_table_ranked.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 155976076 to 155474628
Data columns (total 1111 columns):
prob_default                            float32
prob_fullypaid                          float32
return_preds                            float64
loan_amnt                               int64
funded_amnt                             float64
int_rate                                float64
emp_length                              float64
annual_inc                              float64
dti                                     float64
delinq_2yrs                             int64
earliest_cr_line                        int64
fico_range_low                          int64
fico_range_high                         int64
inq_last_6mths                          int64
mths_since_last_delinq                  float64
mths_since_last_record                  float64
open_acc                                int64
pub_rec                                 int64
revol_bal                       

In [7]:
port_prob_def

0.04148772358894349

In [8]:
port_exp_return

0.09476678749345081

In [9]:
port_shrop_ratio

4.93803789364291

In [10]:
max_investable

161825.0

In [23]:
rec_table_ranked['purpose_car']

id
155976076    0.0
155879545    0.0
155915661    0.0
154588667    0.0
156116626    0.0
155189447    0.0
155466042    0.0
155964253    0.0
155780689    0.0
155601116    0.0
155976413    0.0
155474628    0.0
Name: purpose_car, dtype: float64